# Settings

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import shutil
import sys
import json
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
tqdm.pandas()
import math
import random
import torch

import warnings
warnings.filterwarnings('ignore')

In [2]:
class dummy_class:
    def __init__(self):
        pass
    
args = dummy_class()

args.seed = 42

args.summarization_input = '/train.tsv'
args.summarization_output = '/test.tsv'

args.summarization_pretrain = 'gogamza/kobart-summarization'
# args.summarization_pretrain = 'gogamza/kobart-base-v2'
args.version = 'backup8'
args.summarization_model = f'kobart/{args.version}'
args.summarization_model_b = 'kobart/backup7'
args.isBefore = True

# args.sentence_embedding_pretrain = 'BM-K/KoSimCSE-bert-multitask'
args.sentence_embedding_pretrain = 'BM-K/KoSimCSE-roberta-multitask'

args.check_percent = 0.99
args.max_len = 1024
args.ignore_index = -100

args.num_beams=5
args.max_length=512
args.bos_token_id = 0
args.eos_token_id=1
args.pad_token_id = 3
args.decoder_start_token_id = 0
args.repetition_penalty = 5.0
args.length_penalty = 1.0

args.scale = 'small' # small / middle / original
args.newSample = False

def add_padding_data(inputs):
    if len(inputs) < args.max_len:
        pad = np.array([args.pad_token_id] *(args.max_len - len(inputs)))
        inputs = np.concatenate([inputs, pad])
    else:
        inputs = inputs[:args.max_len]

    return inputs

def filtering_text(text):
    try:
        
        length = len(text)
        index = set(range(length))

        pairs1 = list()
        stack = list()
        for i in range(length):
            if text[i] == '(':
                stack.append(i)
            elif text[i] == ')':
                if stack:
                    pairs1.append((stack.pop(), i))
        
        for i in stack:
            pairs1.append((i, i))
        
        pairs2 = list()
        stack = list()
        for i in range(length):
            if text[i] == '[':
                stack.append(i)
            elif text[i] == ']':
                if stack:
                    pairs2.append((stack.pop(), i))
                    
        for i in stack:
            pairs2.append((i, i))
    
    except:
        print(text)
                
    for start, end in pairs1 + pairs2:
        index -= set(range(start, end+1))

    index = list(index)
    index.sort()
    text = ''.join([text[idx] for idx in index])
            
    return text

os.environ["PYTHONHASHSEED"] = str(args.seed)
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
torch.backends.cudnn.deterministic = True

In [3]:
def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])

In [4]:
path = '.'
kobart_data_path = path + '/kobart/data'
data_path = path + '/data'
patent_path = data_path + '/patent'
patent_training_path = patent_path + '/Training/training_특허전체'
patent_validation_path = patent_path + '/Validation/validation_특허전체'
patent_name = '/특허전체_{}-Y-{}.json'
patent_training_index = list(zip([9] * 33, range(33))) + list(zip([10] * 14, range(14)))
patent_validation_index = list(zip([10] * 5, range(14,20)))
patent_training_name = '/patent_training.tsv'
patent_validation_name = '/patent_validation.tsv'
file_name = '/patent.tsv'
save_name = '/patents_10.tsv'

class_name = '/class.tsv'

# columns = ['순서', '출원번호', '발명의명칭', '출원인', '공개번호', '원출원번호', '발명자', '관련 출원번호', '요약', '청구항']
# pd.DataFrame(columns = columns).to_csv(data_path + file_name, sep = '\t', index = False)

# columns = ['제목', '문서번호', '등록번호', '등록날짜', '요약(Input)', '요약(Output)']
# pd.DataFrame(columns = columns).to_csv(patent_training_path + patent_training_name, sep = '\t', index = False)
# pd.DataFrame(columns = columns).to_csv(patent_validation_path + patent_validation_name, sep = '\t', index = False)

# columns = ['order', 'abstract', 'number', 'class']
# pd.DataFrame(columns = columns).to_csv(data_path + class_name, sep = '\t', index = False)

# Retrieval

In [ ]:
def cal_score(a, b):
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1))

In [19]:
companies_dart = pd.read_csv(data_path + '/dart_industry_code_patent.csv')
companies_smes = pd.read_csv(data_path + '/smes_industry_code_patent.csv')

# companies_dart['출원인']= companies_dart[['bizr_name', 'jurir_name']].apply(lambda x : x[0] if pd.notna(x[0]) else x[1], axis = 1)
# companies_dart['출원인'] = companies_dart['출원인'].apply(filtering_company).apply(lambda x : x.split()[0] if pd.notna(x) else np.nan)
# companies_dart.to_csv(data_path + '/dart_industry_code_patent.csv', index = False)

companies_smes['bizr_no'] = companies_smes['사업자번호'].apply(lambda x : x.replace('-', '') if pd.notna(x) else np.nan)
# companies_smes['출원인'] = companies_smes['bizr_name'].apply(filtering_company).apply(lambda x : x.split()[0] if pd.notna(x) else np.nan)
# companies_smes.to_csv(data_path + '/smes_industry_code_patent.csv', index = False)

In [20]:
companies_smes['회사명'] = companies_smes['회사명'].apply(filtering_company)
companies_dart['stock_name'] = companies_dart['stock_name'].apply(filtering_company)

temp_dart = companies_dart[(companies_dart['stock_name'].notna()) & (companies_dart['bizr_no'].notna())][['stock_name', 'bizr_no']].rename({'stock_name' : '회사명'}, axis = 1)
temp_smes = companies_smes[(companies_smes['회사명'].notna()) & (companies_smes['bizr_no'].notna())][['회사명', 'bizr_no']]

temp_dart = temp_dart[temp_dart['bizr_no'].apply(len) == 10]
temp_smes = temp_smes[temp_smes['bizr_no'].apply(len) == 10]

temp_doc = pd.concat([temp_dart, temp_smes], axis = 0)
temp_doc = temp_doc.groupby('회사명')['bizr_no'].unique().apply(lambda x : x[0]).reset_index()
temp_doc = temp_doc.rename({'bizr_no' : '사업자번호'}, axis = 1)

temp_doc = temp_doc[temp_doc['회사명'] != '']

In [7]:
df_biz = pd.read_csv(data_path + '/patent_companies.tsv', sep = '\t')
number2embedding = dict(zip(np.load(data_path + '/number.npy', allow_pickle = True), np.load(data_path + '/embedding.npy')))
df_biz['embedding'] = df_biz['출원번호'].map(number2embedding)

In [8]:
df_biz

,출원번호,출원일자,발명의명칭,출원인,회사명,사업자번호,원출원번호,abstract,summarized_abstract,embedding
0,10-2023-7020860,2019.08.06,영상 코딩 시스템에서 컨스트럭티드 어파인 MVP 후보를 사용하는 어파인 움직임 예측...,엘지전자,LG전자,107-86-14075,10-2022-7032865,본 발명에 따른 디코딩 장치에 의하여 수행되는 영상 디코딩 방법은 비트스트림으로부터...,"본 발명은 영상 디코딩 방법 및 장치에 관한 것으로서, 보다 상세하게는 비트스트림으...","[0.664092481136322, 0.08804067969322205, -0.37..."
1,10-2023-7015450,2021.11.05,BLE를 이용한 무선 충전 시스템에서 사용자 프라이버시 보호 방법 및 장치(METH...,엘지전자,LG전자,107-86-14075,NaN,본 명세서는 무선 전력 수신 장치에 의해 수행되는 아웃 밴드 통신의 연결을 수행하는...,본 명세서는 무선 전력 전송 장치와 블루투스 기반 통신을 이용하는 아웃 밴드 통신의...,"[-0.00744166923686862, -0.36963945627212524, -..."
2,10-2023-7012720,2021.11.05,NR V2X에서 모드 1 CG 자원에 기반하여 사이드링크 DRX을 지원하는 방법 및...,엘지전자,LG전자,107-86-14075,NaN,"일 실시 예에 있어서, 제 1 장치가 무선 통신을 수행하는 방법이 제안된다. 상기 ...","관련 출원에 대한 상호 참조본 출원은 미국 가특허 출원 제61/842,320호의 우...","[0.027932744473218918, -0.6580889821052551, 0...."
3,10-2023-7007080,2020.09.07,수신기의 신호 진폭 범위에서 AIRCOMP에 대한 바이어스를 사용하여 그래디언트 기...,엘지전자,LG전자,107-86-14075,NaN,수신기의 신호 진폭 범위에서 AirComp에 대한 바이어스를 사용하여 그래디언트 기...,본 발명은 수신기의 신호 진폭 범위에서 AirComp에 대한 바이어스를 사용하여 그...,"[0.20534002780914307, -0.05104814097285271, 0...."
4,10-2023-7007060,2020.08.20,무선 통신 시스템에서 심층 신경망의 연산을 제어하는 방법 및 이를 위한 장치(MET...,엘지전자,LG전자,107-86-14075,NaN,무선 통신 시스템(wireless communication system)에서 단말이...,본 발세서는 무선 통신 시스템에서 단말이 심층 신경망의 연산을 제어하는 방법에 관한...,"[0.6980687379837036, -0.0912511795759201, -0.3..."
...,...,...,...,...,...,...,...,...,...,...
2050442,10-1992-0018556,1992.10.09,볼링핀세터의핀개별제어장치및방법(),대생기업,대생기업,116-81-06948,NaN,본 발명은 볼링핀 세터의 핀개별 제어장치 및 방법에 관한 것으로 볼링핀을 공급하지 ...,본 발명은 볼링핀 세터의 핀개별 제어장치 및 방법에 관한 것이다.,"[-0.18948592245578766, -0.2549843192100525, 0...."
2050443,10-1992-0002146,1992.02.14,그레이스케일폰트의생성방법(METHOD OF GENERATING GREY-SCALE ...,플러스프로핏,플러스프로핏,220-81-22913,NaN,본 발명은 윤곽선 마스터폰트로부터 그레이 스케일 폰트를 직접 생성하는 방법을 개시한...,본 발명은 윤곽선 마스터폰트로부터 그레이 스케일 폰트를 직접 생성하는 방법에 관한 ...,"[0.3320665955543518, -0.193498432636261, -0.50..."
2050444,10-1991-0019874,1991.11.08,습도측정표시장치(),케이씨씨자원개발,케이씨씨자원개발,132-81-05543,NaN,"분말상 고체내의 습도를 측정하여 표시하는 습도측정표시장치로서, 수분의 양을 정확히 ...","본 발명은 습도측정표시장치에 관한 것으로, 더욱 상세하게는 분말상 고체내의 습도를 ...","[0.21788185834884644, -0.17126284539699554, -1..."
2050445,10-1991-0015642,1991.09.05,양모의스케일제거방법(),한신모방,한신모방,605-81-00842,NaN,1. 청구범위에 기재된 발명이 속한 기술분야 양모의 스케일 제거방법 2. 발명이 해...,본 발명은 청구범위에 기재된 발명이 속한 기술분야 양모의 스케일 제거방법에 관한 것...,"[0.06319601833820343, -0.6899250745773315, 0.0..."


In [9]:
df_biz['사업자번호'] = df_biz['사업자번호'].progress_apply(lambda x : x.replace('-', ''))

100%|██████████| 2050447/2050447 [00:02<00:00, 983825.82it/s] 


In [23]:
biz_number = list()
emb_list = list()

for number in tqdm(df_biz['사업자번호'].unique()):
    temp = df_biz.loc[df_biz['사업자번호'] == number, 'embedding']
    if temp.shape[0] > 50000:
        temp = temp.sample(n = 50000)
    emb = torch.tensor(temp.apply(lambda x : x.tolist()).values.tolist())
    index = cal_score(emb, emb).mean(dim = 1).argmax().item()
    
    biz_number.append(number)
    emb_list.append(emb[index].tolist())
    
# np.save(data_path + '/biz_number.npy', biz_number)
# np.save(data_path + '/emb_list.npy', emb_list)

biz2number = dict(zip(biz_number, emb_list))

100%|██████████| 38572/38572 [1:33:58<00:00,  6.84it/s]


In [9]:
biz_number = np.load(data_path + '/biz_number.npy')
emb_list = np.load(data_path + '/emb_list.npy')

biz2emb = dict(zip(biz_number, emb_list))

In [21]:
df_companies = pd.read_csv(data_path + '/smes_naver_concated.csv')
df_companies['회사명'] = df_companies['회사명'].apply(filtering_company)

df_companies = df_companies[df_companies['회사명'] != '']

company2type = df_companies.groupby('회사명')['type'].apply(lambda x : x.values).apply(lambda x : '상장회사' if '상장회사' in x else '비상장회사').reset_index()

df_companies.drop('type', axis = 1, inplace = True)
df_companies = df_companies.merge(company2type, on = '회사명', how = 'left')

df_yes = df_companies[df_companies['type'] == '상장회사'][['회사명', 'concated']].rename({'concated' : '주생산품'}, axis = 1)
df_no = df_companies[df_companies['type'] == '비상장회사'][['회사명', '주생산품']]

df_yes['주생산품'] = df_yes['주생산품'].apply(lambda x : x.replace(',', ' '))
df_yes['주생산품'] = df_yes['주생산품'].apply(lambda x : x.replace('  ', ' '))

df_no['주생산품'] = df_no['주생산품'].apply(filtering_company)

df_yes = df_yes.groupby('회사명')['주생산품'].apply(lambda x : ' '.join(x)).reset_index()
df_no = df_no.groupby('회사명')['주생산품'].apply(lambda x : ' '.join(x)).reset_index()

df_yes = df_yes.merge(temp_doc, on = '회사명') # 사업자번호 있는 경우만 고려함
df_no = df_no.merge(temp_doc, on = '회사명') # 사업자번호 있는 경우만 고려함

df_yes = df_yes.merge(df_companies[['회사명', 'type']].drop_duplicates(), on = '회사명', how = 'left')
df_no = df_no.merge(df_companies[['회사명', 'type']].drop_duplicates(), on = '회사명', how = 'left')

In [13]:
df_yes

,회사명,주생산품,사업자번호,type
0,3S,반도체 재료/부품 기계,1298116215,상장회사
1,AJ네트웍스,도로와철도운송,2148648586,상장회사
2,AK홀딩스,항공/저가 항공사 페인트 화학,1138110894,상장회사
3,APS,OLED IT서비스,2298122962,상장회사
4,AP시스템,OLED 플렉서블 디스플레이 LCD장비 반도체 장비 디스플레이장비및부품,2048700511,상장회사
...,...,...,...,...
2515,흥국에프엔비,음식료업종 음료,1278610281,상장회사
2516,흥국화재,손해보험 손해보험,1028107837,상장회사
2517,흥아해운,해운 해운사,1208162522,상장회사
2518,희림,리모델링/인테리어 건설,2148115867,상장회사


In [14]:
df_no

,회사명,주생산품,사업자번호,type
0,100물류,물류솔루션 및 물류창고 관리업,6198107320,비상장회사
1,123푸드,막창,3658701227,비상장회사
2,1리터,콘텐츠 및 영상물 제작,6178801483,비상장회사
3,1인치,모바일 애플리케이션,6838802043,비상장회사
4,210컴퍼니,"남성여성 일상복, 코트, 셔츠, 팬츠, 티셔츠 등",3038702077,비상장회사
...,...,...,...,...
26186,힘멜,사무용 가구,1288167502,비상장회사
26187,힘찬마루,건강진액,6931900221,비상장회사
26188,힘찬장애인복지회,"수배전반,자동제어반",4098217913,비상장회사
26189,힘테크,OLED용 색소,6198700824,비상장회사


In [22]:
import torch
from transformers import AutoModel, AutoTokenizer
from math import ceil

df_temp = pd.concat([df_yes, df_no], axis = 0).reset_index(drop = True).rename({'주생산품' : 'summarized_abstract'}, axis = 1)

model = AutoModel.from_pretrained(args.sentence_embedding_pretrain).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(args.sentence_embedding_pretrain)

sentences = list(df_temp['summarized_abstract'].values)

total = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

batch_size = 32
outputs = list()

for i in tqdm(range(ceil(len(sentences) / batch_size))):
    batch = dict([(key, total[key][i*batch_size:(i+1)*batch_size].to('cuda')) for key in total.keys()])
    embeddings, _ = model(**batch, return_dict=False)
    for b in range(len(embeddings)):
        outputs.append(embeddings[b][0].detach().cpu().tolist())
        
df_temp['embedding'] = outputs
df_temp

,회사명,summarized_abstract,사업자번호,type,embedding
0,3S,반도체 재료/부품 기계,1298116215,상장회사,"[0.21995459496974945, -0.535297155380249, -0.5..."
1,AJ네트웍스,도로와철도운송,2148648586,상장회사,"[-0.38023561239242554, 0.20949509739875793, -0..."
2,AK홀딩스,항공/저가 항공사 페인트 화학,1138110894,상장회사,"[0.007067902013659477, 0.10890291631221771, 0...."
3,APS,OLED IT서비스,2298122962,상장회사,"[0.26161298155784607, 0.25560083985328674, 0.5..."
4,AP시스템,OLED 플렉서블 디스플레이 LCD장비 반도체 장비 디스플레이장비및부품,2048700511,상장회사,"[-0.05506177246570587, -0.3216588795185089, -0..."
...,...,...,...,...,...
28706,힘멜,사무용 가구,1288167502,비상장회사,"[-0.5627197027206421, 0.05131729692220688, -0...."
28707,힘찬마루,건강진액,6931900221,비상장회사,"[0.4448913335800171, -0.5397737622261047, -0.3..."
28708,힘찬장애인복지회,"수배전반,자동제어반",4098217913,비상장회사,"[-0.0009849313646554947, 0.4955686032772064, -..."
28709,힘테크,OLED용 색소,6198700824,비상장회사,"[-0.2863457500934601, 0.006632595788687468, 0...."


In [43]:
df_yes = df_temp[df_temp['type'] == '상장회사'][['사업자번호', '회사명', 'summarized_abstract', 'embedding']].rename({'summarized_abstract' : '주생산품'}, axis = 1)
df_no = df_temp[df_temp['type'] == '비상장회사'][['사업자번호', '회사명', 'summarized_abstract', 'embedding']].rename({'summarized_abstract' : '주생산품'}, axis = 1)

df_yes['pembedding'] = df_yes['사업자번호'].map(biz2emb)
df_no['pembedding'] = df_no['사업자번호'].map(biz2emb)

df_yes['fembedding'] = df_yes[['embedding', 'pembedding']].apply(lambda x : x[1] if type(x[1]) == np.ndarray else x[0], axis = 1)
df_no['fembedding'] = df_no[['embedding', 'pembedding']].apply(lambda x : x[1] if type(x[1]) == np.ndarray else x[0], axis = 1)

df_yes.drop(['embedding', 'pembedding'], axis = 1, inplace = True)
df_no.drop(['embedding', 'pembedding'], axis = 1, inplace = True)

df_yes = df_yes.rename({'fembedding' : 'embedding'}, axis = 1)
df_no = df_no.rename({'fembedding' : 'embedding'}, axis = 1)

df_yes['embedding'] = df_yes['embedding'].progress_apply(lambda x : list(x))
df_no['embedding'] = df_no['embedding'].progress_apply(lambda x : list(x))

100%|██████████| 26191/26191 [00:01<00:00, 14504.60it/s]


In [44]:
df_yes

,사업자번호,회사명,주생산품,embedding
0,1298116215,3S,반도체 재료/부품 기계,"[0.09916598349809647, -0.9447956085205078, -0...."
1,2148648586,AJ네트웍스,도로와철도운송,"[-0.4880618155002594, -0.2739177942276001, 0.1..."
2,1138110894,AK홀딩스,항공/저가 항공사 페인트 화학,"[-0.06561337411403656, -0.8500086069107056, 0...."
3,2298122962,APS,OLED IT서비스,"[0.14046378433704376, -0.8581822514533997, -0...."
4,2048700511,AP시스템,OLED 플렉서블 디스플레이 LCD장비 반도체 장비 디스플레이장비및부품,"[0.09121190756559372, -0.7881274819374084, 0.1..."
...,...,...,...,...
2515,1278610281,흥국에프엔비,음식료업종 음료,"[-1.0794146060943604, -0.4340958297252655, -0...."
2516,1028107837,흥국화재,손해보험 손해보험,"[0.014492110349237919, -0.5155692100524902, 0...."
2517,1208162522,흥아해운,해운 해운사,"[-0.534527599811554, 0.5391607880592346, 0.078..."
2518,2148115867,희림,리모델링/인테리어 건설,"[-0.2897093892097473, -0.2710621654987335, 0.0..."


In [26]:
df_no

,사업자번호,회사명,주생산품,embedding
2520,6198107320,100물류,물류솔루션 및 물류창고 관리업,"[0.04717600345611572, 0.6353971362113953, -0.2..."
2521,3658701227,123푸드,막창,"[-0.3223477303981781, -0.4957203269004822, -0...."
2522,6178801483,1리터,콘텐츠 및 영상물 제작,"[0.42633312940597534, 0.6746550798416138, -0.7..."
2523,6838802043,1인치,모바일 애플리케이션,"[-0.015885766595602036, -0.3951790928840637, 0..."
2524,3038702077,210컴퍼니,"남성여성 일상복, 코트, 셔츠, 팬츠, 티셔츠 등","[0.23228032886981964, -0.7313899397850037, 0.3..."
...,...,...,...,...
28706,1288167502,힘멜,사무용 가구,"[-0.5627197027206421, 0.05131729692220688, -0...."
28707,6931900221,힘찬마루,건강진액,"[0.4448913335800171, -0.5397737622261047, -0.3..."
28708,4098217913,힘찬장애인복지회,"수배전반,자동제어반","[0.14083591103553772, -0.2899269461631775, 0.3..."
28709,6198700824,힘테크,OLED용 색소,"[-0.4274995028972626, -0.6348707675933838, -0...."


In [46]:
biz_number = list()
emb_list = list()

for number in tqdm(df_biz['사업자번호'].unique()):
    temp = df_biz.loc[df_biz['사업자번호'] == number, 'embedding']
    if temp.shape[0] > 50000:
        temp = temp.sample(n = 50000)
    emb = torch.tensor(temp.apply(lambda x : x.tolist()).values.tolist())
    index = cal_score(emb, emb).mean(dim = 1).argmax().item()
    
    biz_number.append(number)
    emb_list.append(emb[index].tolist())
    
# np.save(data_path + '/biz_number.npy', biz_number)
# np.save(data_path + '/emb_list.npy', emb_list)

biz2number = dict(zip(biz_number, emb_list))

In [87]:
yes_biz = df_yes['사업자번호'].values
yes_comp = df_yes['회사명'].values
yes_emb = torch.tensor(df_yes['embedding'].tolist(), dtype = torch.float)

no_biz = list()
no_comp = list()
no_item = list()
no_biz_list = list()
no_comp_list = list()

for biz_number, company, item, emb in tqdm(df_no.values):
    emb = torch.tensor(emb, dtype = torch.float)
    
    no_biz.append(biz_number)
    no_comp.append(company)
    no_item.append(item)
    indices = cal_score(emb, yes_emb).squeeze().argsort(descending=True)
    no_biz_list.append(yes_biz[indices].tolist())
    no_comp_list.append(yes_comp[indices].tolist())

100%|██████████| 26191/26191 [00:26<00:00, 998.56it/s] 


In [159]:
# pd.DataFrame([no_biz, no_comp, no_item, no_biz_list, no_comp_list]).T.rename({0 : '사업자번호_비상장', 1 : '회사명_비상장', 2 : '주생산품_비상장', 3 : '사업자번호_상장', 4 : '회사명_상장'}, axis = 1).to_csv(data_path + '/company_mapping.tsv', sep = '\t', index = False)
pd.read_csv(data_path + '/company_mapping.tsv', sep = '\t')

,사업자번호_비상장,회사명_비상장,주생산품_비상장,사업자번호_상장,회사명_상장
0,6198107320,100물류,물류솔루션 및 물류창고 관리업,"['1388101642', '1308116025', '3018155805', '50...","['선광', '한익스프레스', '동방', '삼일', '인터지스', '태웅로직스', ..."
1,3658701227,123푸드,막창,"['4048112594', '1088161052', '4018106526', '60...","['우리손에프앤지', '농심홀딩스', '동우팜투테이블', 'CJ프레시웨이', '삼화..."
2,6178801483,1리터,콘텐츠 및 영상물 제작,"['2118707434', '1208729530', '1208180345', '12...","['쇼박스', 'NEW', '팬엔터테인먼트', '키이스트', '스튜디오드래곤', '..."
3,6838802043,1인치,모바일 애플리케이션,"['5048164581', '6098131139', '1298138446', '10...","['엔피디', '블루콤', '피델릭스', '컴투스', '유안타증권', '데브시스터즈..."
4,3038702077,210컴퍼니,"남성여성 일상복, 코트, 셔츠, 팬츠, 티셔츠 등","['1208727416', '1108149202', '2018112618', '21...","['쌍방울', '진양화학', '깨끗한나라', '디케이앤디', '코오롱', '효성티앤..."
...,...,...,...,...,...
26186,1288167502,힘멜,사무용 가구,"['1358102448', '2068641898', '3108120896', '21...","['현대리바트', '오하임아이엔티', '진영', '퍼시스', '태양', '시디즈',..."
26187,6931900221,힘찬마루,건강진액,"['1308692561', '4068104241', '2068601066', '11...","['루트락', '비엘헬스케어', '대상홀딩스', '유투바이오', '에이치피오', '..."
26188,4098217913,힘찬장애인복지회,"수배전반,자동제어반","['1238625235', '1308106245', '6038103203', '12...","['지투파워', '에이엔피', '제일전기공업', '웨이브일렉트로', '일진전기', ..."
26189,6198700824,힘테크,OLED용 색소,"['1018632319', '1358198089', '5138118626', '13...","['SK이노베이션', '와이솔', '비케이홀딩스', '아모그린텍', '야스', 'L..."
